In [1]:
import transformers, tokenizers, torch
print("torch", torch.__version__)
print("transformers", transformers.__version__)
print("tokenizers", tokenizers.__version__)

torch 2.10.0+cu128
transformers 4.57.6
tokenizers 0.22.2


# **Augmentation 적용**

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
model_id = 'Bllossom/llama-3.2-Korean-Bllossom-AICA-5B'
# model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# 하단의 "가중치 로딩 리포트(경고)"는 텍스트만 생성할 경우 무시해도 됨

`torch_dtype` is deprecated! Use `dtype` instead!


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## **텍스트 생성함수**

In [3]:
def textAugmentation(model, PROMPT, text, instruction):
    import time
    import torch

    # ✅ CUDA 있을 때만 동기화 (CPU 환경에서도 안전)
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    start_time = time.time()

    instruction = f"{instruction}\n\n{text}"

    messages = [
        {"role": "system", "content": PROMPT},
        # {"role": "user", "content": FEWSHOT},
        {"role": "user", "content": instruction},
    ]

    # apply_chat_template 결과가 환경/버전에 따라 Tensor 또는 BatchEncoding(dict)일 수 있음
    enc = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # ✅ enc 타입에 따라 input_ids / attention_mask / prompt_len 처리
    if isinstance(enc, torch.Tensor):
        input_ids = enc
        attention_mask = None
        prompt_len = enc.shape[-1]
    else:
        input_ids = enc["input_ids"]
        attention_mask = enc.get("attention_mask", None)
        prompt_len = enc["input_ids"].shape[-1]

    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,

        # 길이 제어
        max_new_tokens=512,

        # 샘플링 설정
        do_sample=True,
        temperature=0.75,
        top_p=0.9,

        # 반복 방지
        repetition_penalty=1.10,

        # 종료 / 패딩
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id
    )

    result = tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True)

    # ✅ CUDA 있을 때만 동기화
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    elapsed = time.time() - start_time
    print(f"⏱ GPU 포함 실행 시간: {elapsed:.3f}초")

    return result


## **프롬프트 작성**

In [4]:
PROMPT = """
너는 한국어 데이터 Augmentation 전문가야. 아래의 규칙에 따라 문장을 변형해야해

규칙:
1. 설명, 해설, 제목 없이 무조건 결과만 출력한다.
2. 결과는 반드시 한국어로 작성한다.
3. 상황과 핵심 의미는 유지한다.
4. 각 문장은 반드시 어휘/어순/표현을 바꿔 쓴다.
5. 각 문장은 원문보다 짧게 작성한다.
6. 각 문장은 절대 똑같이 사용하지 않는다. 

"""

# FEWSHOT = """
# [예시 1]
# 입력: 오늘 회의가 너무 길어서 점심을 제대로 못 먹었어.
# 출력: 회의가 길어져서 점심을 거의 거르게 됐어.

# [예시 2]
# 입력: 비가 계속 내려서 밖에 나가기 싫다.
# 출력: 계속 비가 와서 외출할 마음이 안 들어.

# [예시 3]
# 입력: 숙제가 많아서 밤늦게까지 공부했다.
# 출력: 과제가 쌓여 늦은 밤까지 공부했어.
# """

instruction = "아래의 한국어 문장을 규칙에 따라서 Augmentation 해줘."

In [5]:
# 테스트
text = "35번 손님 아이스커피 두잔나왔습니다 아이스커피? 네 맛있게드세요 저기요 아가씨 나는 아아스 시킨적이 없는데? 아 분명 오늘 날이 더우시다고 아이스로 시키셨는데요 내가 그랫어? 네분명히. 아그런 기억이 없는데? 아가씨가잘못안거 아니야? 아니요. 오늘 손님이 첫 주문이라 확실히 기억하고 있습니다 아가씨. 왜이렇게 유도리가 없이 굴어 그냥 아 제가 잘못 주문 받았습니다 하면 되지? .네?. 어휴 유도리 없어 그냥 마실게"
print(text)
print("-------------------------------")
print(
    textAugmentation(
        model=model,
        PROMPT=PROMPT,
        # FEWSHOT=FEWSHOT,
        text=text,
        instruction=instruction
    )
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


35번 손님 아이스커피 두잔나왔습니다 아이스커피? 네 맛있게드세요 저기요 아가씨 나는 아아스 시킨적이 없는데? 아 분명 오늘 날이 더우시다고 아이스로 시키셨는데요 내가 그랫어? 네분명히. 아그런 기억이 없는데? 아가씨가잘못안거 아니야? 아니요. 오늘 손님이 첫 주문이라 확실히 기억하고 있습니다 아가씨. 왜이렇게 유도리가 없이 굴어 그냥 아 제가 잘못 주문 받았습니다 하면 되지? .네?. 어휴 유도리 없어 그냥 마실게
-------------------------------
⏱ GPU 포함 실행 시간: 5.656초
아가씨 오랜만에 왔어요. 아이스커피를 자주 드시나요? 네, 맛있게 드세요. 저기요? 아 가씨 나은 아아스 시킨 적이 없어요. 아 분명 오늘 날이 더웠다고 아이스로 시켰는데요, 내가 그랬어? 네 분명히. 아, 그런 기억이 없어요? 아가씨가 잘못 말씀쳤나요? 아니요. 오늘 손님이 첫 주문이었어요. 확실히 기억하고 있어요, 아가씨. 왜 이렇게 유도리가 없이 굴었어요? 그냥 마실게.


In [6]:
# [직장 내 괴롭힘] 테스트
text = "나이가 몇인데 내가 이런 것까지 일러줘야돼? 죄송합니다 기본적인 것도 탑재가 안되있으면 어떡해? 죄송합니다. 그 죄송하다는 소리좀 듣기 싫어 제대로 할수 없어? 다음부터 제대로 해보겠습니다 그따위로 하다간 다음은 없을지도 몰라 제대로좀해 제발! 네 정말 죄송합니다 아주 가정에서부터 교육을 다시 받아야돼 문제가 많아 요새 젊은 것들은 . 죄송합니다"
print(text)
print("-------------------------------")
print(
    textAugmentation(
        model=model,
        PROMPT=PROMPT,
        # FEWSHOT=FEWSHOT,
        text=text,
        instruction=instruction
    )
)

나이가 몇인데 내가 이런 것까지 일러줘야돼? 죄송합니다 기본적인 것도 탑재가 안되있으면 어떡해? 죄송합니다. 그 죄송하다는 소리좀 듣기 싫어 제대로 할수 없어? 다음부터 제대로 해보겠습니다 그따위로 하다간 다음은 없을지도 몰라 제대로좀해 제발! 네 정말 죄송합니다 아주 가정에서부터 교육을 다시 받아야돼 문제가 많아 요새 젊은 것들은 . 죄송합니다
-------------------------------
⏱ GPU 포함 실행 시간: 4.094초
안녕하세요, 나이 몇 살까요? 이렇게 일을 일러주면 너무 죄송해요. 기본적인 것도 잘 하지 않겠네요, 죄송해요. 그 말에 듣기 싫어요, 제대로 할 수 없어? 다음번엔 제대로 해볼게요. 그런데 두고 가면 다음에는 없을지도 모르겠어요. 제대로 좀 해주세요, 제발요. 진짜 죄송해요. 학교에서부터 다시 시작해서 괜찮겠어요? 요새's 젊은이들'도 많이 문제일 거예요, 죄송해요.


In [7]:
# [협박 대화] 테스트
text = "저기요 할아버지 여기 버스 안이기도 한데 환기도 잘 안되니까 마스크 좀 써주세요. 너가 뭔데 나보고 쓰라 말아야? 나한테 뭐 해준거 있어? 그게 아니고 이제 버스에서 원래 마스크 안쓰면 내리시거나 벌금 내셔야 해요. 그래서 뭐 어쩌라고. 지금 나 내좇을 수 있는 사람 있어? 그럼 여기서 내리세요 제가 내려드릴게요. 공공장소인데 남한테 피해주면 안되시죠. 너 내가 죽여버릴거야. 각오해. 마스크? 그거 평생 안 쓰게 만들어줄게. 그게 뭔 소리세요. 그런 말 함부러 하시면 안되죠. 아무리 연세가 많으시다고 해도. 그니까 애초에 왜 내 신경을 건드려. 내가 지금은 이렇게 조용하지만 어릴적에는 여럿 패고 다녔어. 오랜만에 힘 좀 써야하네. 협박하시면 신고하실거고요. 마스크 하나 써달라고 했다고 그렇게 말씀하시지 마세요. 내가 뭐 어떻게 말하던 무슨 상관인가? 실제로 죽일것도 아닌데. 오늘 저녁에 죽을거 같을 때 신나게 신고하던지 끌끌"
print(text)
print("-------------------------------")
print(
    textAugmentation(
        model=model,
        PROMPT=PROMPT,
        # FEWSHOT=FEWSHOT,
        text=text,
        instruction=instruction
    )
)

저기요 할아버지 여기 버스 안이기도 한데 환기도 잘 안되니까 마스크 좀 써주세요. 너가 뭔데 나보고 쓰라 말아야? 나한테 뭐 해준거 있어? 그게 아니고 이제 버스에서 원래 마스크 안쓰면 내리시거나 벌금 내셔야 해요. 그래서 뭐 어쩌라고. 지금 나 내좇을 수 있는 사람 있어? 그럼 여기서 내리세요 제가 내려드릴게요. 공공장소인데 남한테 피해주면 안되시죠. 너 내가 죽여버릴거야. 각오해. 마스크? 그거 평생 안 쓰게 만들어줄게. 그게 뭔 소리세요. 그런 말 함부러 하시면 안되죠. 아무리 연세가 많으시다고 해도. 그니까 애초에 왜 내 신경을 건드려. 내가 지금은 이렇게 조용하지만 어릴적에는 여럿 패고 다녔어. 오랜만에 힘 좀 써야하네. 협박하시면 신고하실거고요. 마스크 하나 써달라고 했다고 그렇게 말씀하시지 마세요. 내가 뭐 어떻게 말하던 무슨 상관인가? 실제로 죽일것도 아닌데. 오늘 저녁에 죽을거 같을 때 신나게 신고하던지 끌끌
-------------------------------
⏱ GPU 포함 실행 시간: 8.000초
아래의 한국어 문장을 규칙에 따라서 Augmentation 해줘.

저기요 할아버지 여기 버스 안이기도 한데 환기도 잘 안되니까 마스크 좀 써주세요. 너가 뭔데 나보고 쓰라 말아야? 

마스크? 그거 평생 안 쓰게 만들어줄게. 

그럼 여기서 내리세요 제가 내려드릴게요. 

공공장소인데 남한테 피해주면 안되시죠. 

아무리 연세가 많으시다고 해도. 

그니까 애초에 왜 내 신경을 건드려. 

내가 지금은 이렇게 조용하지만 어랬다면 여럿 패고 다녔어. 

오랜만에 힘 좀 써야하네. 

협박하시면 신고하실거고요. 

마스크 하나 써달라고 했다고 이런 식으로 말하니.

내가 뭐 어떻게 말하던 이 상관이 뭐야?

사실로 죽일려고 하지 않는데. 

오늘 저녁에 죽을 것 같았으니 신고할 거야.


In [8]:


# [기타 괴롭힘 대화] 테스트
text = "어이 아가씨!! 저 아세요.? 이쁜데 어디 살아? 저리 가세요. 아 비싸게 굴지 말고. 몸매 죽이는데? 따라오지 마세요!! 아 그러지 말고 나 좀 따라와 봐. 재밌는 거 보여줄게. 따라오지 마시라고요!!! 더럽게 비싸게 구네!! 좀 따라오라고!! 이거 놔요!!!!!"
print(text)
print("-------------------------------")
print(
    textAugmentation(
        model=model,
        PROMPT=PROMPT,
        # FEWSHOT=FEWSHOT,
        text=text,
        instruction=instruction
    )
)

어이 아가씨!! 저 아세요.? 이쁜데 어디 살아? 저리 가세요. 아 비싸게 굴지 말고. 몸매 죽이는데? 따라오지 마세요!! 아 그러지 말고 나 좀 따라와 봐. 재밌는 거 보여줄게. 따라오지 마시라고요!!! 더럽게 비싸게 구네!! 좀 따라오라고!! 이거 놔요!!!!!
-------------------------------
⏱ GPU 포함 실행 시간: 3.606초
어이! 아가씨! 저(A가씨) 여기(여기) 살아요? 저리 가세요. 아 비싸게 굴지 말고. 몸매 죽이는데요? 따라오지 마세요! 아 그러지 말고 나(A가씨)를 좀 따라와 봐예요. 재밌는 게 보여줄게요. 따라오지 마세요! 더럽게 비싸게 구었어요. 좀 따라오라고 해요! 이거 놔요!


In [9]:


# [갈취 대화] 테스트
text = "어이 거기 예?? 너 말이야 너. 이리 오라고 무슨 일. 너 옷 좋아보인다? 얘 돈 좀 있나봐 아니에요.돈 없어요 뒤져서 나오면 넌 죽는다 오늘 피시방 콜? 콜. 마지막 기회다. 있는거 다 내놔 정말 없어요"
print(text)
print("-------------------------------")
print(
    textAugmentation(
        model=model,
        PROMPT=PROMPT,
        # FEWSHOT=FEWSHOT,
        text=text,
        instruction=instruction
    )
)

어이 거기 예?? 너 말이야 너. 이리 오라고 무슨 일. 너 옷 좋아보인다? 얘 돈 좀 있나봐 아니에요.돈 없어요 뒤져서 나오면 넌 죽는다 오늘 피시방 콜? 콜. 마지막 기회다. 있는거 다 내놔 정말 없어요
-------------------------------
⏱ GPU 포함 실행 시간: 2.283초
어이 거기 예? 너 말이야 너. 이리 오라고 무슨 일. 너 옷 좋아보인다? 얘 돈 좀 있나봐 아니에요. 돈 없어요 뒤겨서 나오면 넌 죽는다 오늘 피시방 콜? 콜. 마지막 기회다. 있는거 다 내놔 정말 없어요
